In [2]:
import networkx as nx

# Part 1

Idea: Problem is finding the maximum path sum in a tree that represents the sequential actions taken per minute.

- create a DAG of actions per blueprint with networkx, get all paths of length 24 and calculate highest score

In [8]:
from dataclasses import dataclass
from enum import Enum

class Robot(Enum):
    ORE = 1
    CLAY = 2
    OBSIDIAN = 3
    GEODE = 4

ROBOT_TYPES = [Robot.ORE, Robot.CLAY, Robot.OBSIDIAN, Robot.GEODE]

@dataclass
class Blueprint:
    # cost only ore
    ore_robot_cost: int
    # cost only ore
    clay_robot_cost: int
    # cost ore and clay
    obsidian_robot_cost: tuple[int,int]
    # cost ore and obsidian
    geode_robot_cost: tuple[int,int]

@dataclass
class State:
    ore:int
    ore_robots: int
    clay: int
    clay_robots: int
    obsidian: int
    obsidian_robots: int
    geodes: int
    geode_robots: int

    def can_build(self, blueprint: Blueprint, robot: Robot):
        match robot:
            case(Robot.ORE): return self.ore >= blueprint.ore_robot_cost
            case(Robot.CLAY):  return self.clay >= blueprint.clay_robot_cost
            case(Robot.OBSIDIAN): 
                ore, clay = blueprint.obsidian_robot_cost
                return (self.ore >= ore and self.clay >= clay)
            case(Robot.GEODE): 
                ore, obs = blueprint.geode_robot_cost
                return (self.ore >= ore and self.obsidian >= obs)

    def advance(self, minutes=1):
        for _ in range(minutes):
            self.ore += self.ore_robots
            self.clay += self.clay_robots
            self.obsidian += self.obsidian_robots
            self.geodes += self.geode_robots

In [10]:
s = State(0,1,0,1,0,1,0,1)
s.advance(3)
s

State(ore=3, ore_robots=1, clay=3, clay_robots=1, obsidian=3, obsidian_robots=1, geodes=3, geode_robots=1)